1. Imports

In [ ]:
import torch
from torch import nn, optim
from torch.utils.data import DataLoader, Subset
from torchvision import datasets, transforms
from opacus import PrivacyEngine
import numpy as np

2. Model Definition

In [ ]:
class SimpleCNN(nn.Module):
    def __init__(self):
        super().__init__()
        self.features = nn.Sequential(
            nn.Conv2d(1, 32, 3, 1),
            nn.ReLU(),
            nn.Conv2d(32, 64, 3, 1),
            nn.ReLU(),
            nn.Flatten(),
        )
        sample_input = torch.zeros(1, 1, 28, 28)
        sample_output = self.features(sample_input)
        num_features = sample_output.shape[1]

        self.classifier = nn.Sequential(
            nn.Linear(num_features, 128),
            nn.ReLU(),
            nn.Linear(128, 10),
        )
    
    def forward(self, x):
        x = self.features(x)
        x = self.classifier(x)
        return x

3. Dataset Definition

In [ ]:
transform = transforms.Compose([transforms.ToTensor()])
train_dataset = datasets.MNIST(root="./data", train=True, download=True, transform=transform)
test_dataset = datasets.MNIST(root="./data", train=False, download=True, transform=transform)


4. Define Modified Dataset and Loaders

In [ ]:
train_dataset_removed = Subset(train_dataset, list(range(1, len(train_dataset))))

train_loader_full = DataLoader(train_dataset, batch_size=64, shuffle=True)
train_loader_removed = DataLoader(train_dataset_removed, batch_size=64, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=1000, shuffle=False)

5. Define Model Training and Predictions

In [ ]:
def train(model, dataloader, epsilon, delta=1e-5, epochs=1):
    optimizer = optim.SGD(model.parameters(), lr=0.01, momentum=0.9)
    criterion = nn.CrossEntropyLoss()

    privacy_engine = PrivacyEngine()
    model, optimizer, dataloader = privacy_engine.make_private(
        module=model,
        optimizer=optimizer,
        data_loader=dataloader,
        noise_multiplier=1.0,
        max_grad_norm=1.0,
    )
    
    model.train()
    for epoch in range(epochs):
        for data, target in dataloader:
            optimizer.zero_grad()
            output = model(data)
            loss = criterion(output, target)
            loss.backward()
            optimizer.step()
    return model
    
def get_predictions(model, dataloader):
    model.eval()
    preds = []
    with torch.no_grad():
        for data, _ in dataloader:
            output = model(data)
            preds.append(output.softmax(dim=1).cpu().numpy())
    return np.vstack(preds)

6. Model Training


model_full = SimpleCNN()
model_full = train(model_full, train_loader_full, epsilon=1.0)

model_removed = SimpleCNN()
model_removed = train(model_removed, train_loader_removed, epsilon=1.0)


7. Comparison of Outputs 

In [ ]:
preds_full = get_predictions(model_full, test_loader)
preds_removed = get_predictions(model_removed, test_loader)

# Compute average absolute difference in predictions
difference = np.abs(preds_full - preds_removed).mean()
print(f"Average difference in predictions: {difference:.6f}")